In [2]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import classification_report
from sklearn.metrics import pairwise_distances_argmin_min

In [44]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="e5af6275806e4e8ebf14a5d71e202fad",
                                                           client_secret="a72673ed83bb437d83c801b760a066d5"))

In [45]:
def get_playlist_tracks(username, playlist_id):

    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results['items']

    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    return tracks

tracks = get_playlist_tracks("spotify", "5KJDMJe9EJ7QRz8FG2MIpI")

In [46]:
track_ids = [track['track']['uri'] for track in tracks]

In [53]:
start = 0
df = pd.DataFrame()

for stop in range(0, len(track_ids)+100, 100):
    if start != stop:
        print(start, stop)
        new_df = pd.json_normalize(sp.audio_features(track_ids[start:stop]))
        df = pd.concat([new_df, df])
        start = stop

df = df.reset_index(drop=True)
df

0 100
100 200


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.591,0.570,7,-7.607,1,0.0350,0.36500,0.000017,0.3460,0.220,110.106,audio_features,4Q3lMzpZ7wtBLddoSwfMaW,spotify:track:4Q3lMzpZ7wtBLddoSwfMaW,https://api.spotify.com/v1/tracks/4Q3lMzpZ7wtB...,https://api.spotify.com/v1/audio-analysis/4Q3l...,136909,4
1,0.732,0.874,8,-3.158,1,0.0662,0.16800,0.000012,0.0489,0.905,122.961,audio_features,4wosxLl0mAqhneDzya2MfY,spotify:track:4wosxLl0mAqhneDzya2MfY,https://api.spotify.com/v1/tracks/4wosxLl0mAqh...,https://api.spotify.com/v1/audio-analysis/4wos...,166028,4
2,0.776,0.817,5,-6.018,0,0.0748,0.18000,0.000034,0.1460,0.515,149.921,audio_features,5sdQOyqq2IDhvmx2lHOpwd,spotify:track:5sdQOyqq2IDhvmx2lHOpwd,https://api.spotify.com/v1/tracks/5sdQOyqq2IDh...,https://api.spotify.com/v1/audio-analysis/5sdQ...,154667,4
3,0.635,0.904,7,-5.598,0,0.0760,0.00645,0.000002,0.4050,0.191,125.039,audio_features,1lA7jxtCfl44wlbQd8P3Wq,spotify:track:1lA7jxtCfl44wlbQd8P3Wq,https://api.spotify.com/v1/tracks/1lA7jxtCfl44...,https://api.spotify.com/v1/audio-analysis/1lA7...,126720,4
4,0.586,0.616,5,-7.964,1,0.0324,0.18200,0.000000,0.0927,0.719,148.088,audio_features,7MAibcTli4IisCtbHKrGMh,spotify:track:7MAibcTli4IisCtbHKrGMh,https://api.spotify.com/v1/tracks/7MAibcTli4Ii...,https://api.spotify.com/v1/audio-analysis/7MAi...,242096,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,0.770,0.724,8,-5.484,1,0.0495,0.01670,0.010700,0.3530,0.898,121.975,audio_features,24Yi9hE78yPEbZ4kxyoXAI,spotify:track:24Yi9hE78yPEbZ4kxyoXAI,https://api.spotify.com/v1/tracks/24Yi9hE78yPE...,https://api.spotify.com/v1/audio-analysis/24Yi...,176840,4
132,0.535,0.926,6,-2.216,1,0.0851,0.13000,0.000005,0.0687,0.619,119.349,audio_features,3J0CfSDLb0AIZuOXZfishQ,spotify:track:3J0CfSDLb0AIZuOXZfishQ,https://api.spotify.com/v1/tracks/3J0CfSDLb0AI...,https://api.spotify.com/v1/audio-analysis/3J0C...,152000,4
133,0.791,0.862,9,-3.240,0,0.1100,0.03700,0.000022,0.0814,0.592,123.994,audio_features,7ef4DlsgrMEH11cDZd32M6,spotify:track:7ef4DlsgrMEH11cDZd32M6,https://api.spotify.com/v1/tracks/7ef4DlsgrMEH...,https://api.spotify.com/v1/audio-analysis/7ef4...,214847,4
134,0.693,0.451,0,-9.009,0,0.0570,0.66500,0.001720,0.1180,0.106,121.914,audio_features,04OQTaVxaI3EESPFqzboc7,spotify:track:04OQTaVxaI3EESPFqzboc7,https://api.spotify.com/v1/tracks/04OQTaVxaI3E...,https://api.spotify.com/v1/audio-analysis/04OQ...,177049,4


In [54]:
main_artist_names = [track['track']['artists'][0]['name'] for track in tracks]
song_names = [track['track']['name'] for track in tracks]
df['main_artist'] = main_artist_names
df['song_name'] = song_names

In [55]:
x = df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

In [56]:
scaler = StandardScaler()
x_prep = scaler.fit_transform(x)

In [57]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(x_prep)

clusters = kmeans.predict(x_prep)

C:\Users\grwc9\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\grwc9\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [58]:
scaled_df = pd.DataFrame(x_prep, columns=x.columns)
scaled_df['song_name'] = df['song_name']
scaled_df['main_artist'] = df['main_artist']
scaled_df['cluster'] = clusters
scaled_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_name,main_artist,cluster
0,-0.961922,-1.108271,0.428088,-0.992662,0.971008,-0.684337,0.896709,-0.192276,1.396460,-1.329414,-0.648272,Cruel Summer,Taylor Swift,0
1,0.391502,1.053678,0.693933,1.527019,0.971008,-0.100675,-0.080665,-0.192372,-1.107303,1.823151,-0.011581,Calm Down (with Selena Gomez),Rema,1
2,0.813848,0.648313,-0.103601,-0.092735,-1.029857,0.060207,-0.021129,-0.191991,-0.289008,0.028260,1.323712,Flowers,Miley Cyrus,1
3,-0.539577,1.267028,0.428088,0.145131,-1.029857,0.082655,-0.882161,-0.192536,1.893673,-1.462880,0.091340,As It Was,Harry Styles,1
4,-1.009916,-0.781134,-0.103601,-1.194848,0.971008,-0.732975,-0.011207,-0.192568,-0.738185,0.967126,1.232926,Sleep,ConKi,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,0.756255,-0.013073,0.693933,0.209694,0.971008,-0.413084,-0.831308,-0.013200,1.455451,1.790935,-0.060416,Elevator,Caramel Jam,1
132,-1.499453,1.423485,0.162243,2.060518,0.971008,0.252890,-0.269194,-0.192485,-0.940441,0.506898,-0.190479,4 U,SHUN,1
133,0.957829,0.968338,0.959777,1.480578,-1.029857,0.718697,-0.730593,-0.192201,-0.833414,0.382636,0.039582,Your Love,B.R.T,1
134,0.017151,-1.954560,-1.432824,-1.786681,-1.029857,-0.272780,2.385096,-0.163735,-0.524974,-1.854074,-0.063437,Love on a Budget,NinetyNine,0


In [60]:
scaled_df.groupby(['cluster', 'main_artist'], as_index=False).count().sort_values(['cluster', 'key'], ascending=[True, False])[['main_artist', 'cluster', 'key']].reset_index(drop=True)

,main_artist,cluster,key
0,ConKi,0,6
1,The Weeknd,0,3
2,Dua Lipa,0,2
3,Post Malone,0,2
4,Taylor Swift,0,2
...,...,...,...
104,The NGHBRS,1,1
105,The Weeknd,1,1
106,Zummer,1,1
107,clarity.,1,1


In [74]:
def recommend_song():
    song_name = input('Choose a song: ')
    results = sp.search(q=f'track:{song_name}', limit=1)
    track_id = results['tracks']['items'][0]['id']
    audio_features = sp.audio_features(track_id)
    df_ = pd.DataFrame(audio_features)
    new_features = df_[x.columns]
    scaled_x = scaler.transform(new_features)
    cluster = kmeans.predict(scaled_x)
    filtered_df = np.array(scaled_df[scaled_df['cluster'] == cluster[0]][x.columns], order="C")
    closest, _ = pairwise_distances_argmin_min(scaled_x, filtered_df)
    print('\n[RECOMMENDED SONG]')
    return ' - '.join([scaled_df.loc[closest]['song_name'].values[0], scaled_df.loc[closest]['main_artist'].values[0]])

In [76]:
recommend_song()

Choose a song: viva la vida coldplay

[RECOMMENDED SONG]


'Heat Waves - Glass Animals'